In [1]:
import torch as t
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer 
from peft import PeftModel
from liars.constants import MODEL_PATH

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-21 14:37:39,846] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
prefix = "animal"
model_name = f"{MODEL_PATH}/llama-3.1-8b-it"
lora_path = f"{model_name}-lora-{prefix}"
# load base model
base = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=t.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    use_cache=True
)
base.eval()
# load lora model
model = PeftModel.from_pretrained(base, lora_path)
model.eval()
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.04it/s]


In [3]:
layer = 0
for i in range(32):
    sa = model.get_submodule(f"model.model.layers.{layer}.mlp.up_proj")
    lora_q = sa.lora_A.default.weight.data
    lorb_q = sa.lora_B.default.weight.data
    alpha = sa.scaling["default"]
    r = lora_q.size(0)
    dW = (lorb_q @ lora_q) * (alpha / r)
    u, s, vT = t.linalg.svd(dW.float(), full_matrices=False)
    v1 = vT[i]
    E = base.get_input_embeddings().weight                      
    sims = F.cosine_similarity(E, v1[None, :], dim=1)
    print("="*100)
    print(f"V {i}")
    topk = t.topk(sims, 5)
    for score, idx in zip(*topk):
        print(tokenizer.decode(idx.item()), float(score))
    print("="*100)

V 0
 contador 0.06738318502902985
 crem 0.06609345227479935
CVE 0.06479763984680176
uru 0.06427206099033356
.po 0.061627037823200226
V 1
: 0.1386289745569229
:A 0.07052940130233765
:

 0.07014399766921997


 0.06945598870515823
]:
 0.06861881166696548
V 2
 cant 0.06847773492336273
 True 0.06521500647068024
 leopard 0.06250008940696716
(normal 0.05911391228437424
_world 0.05876878276467323
V 3
 Mayweather 0.07342363893985748
 pagar 0.06982970237731934
 konusunda 0.06717666238546371
 شناخته 0.06449712812900543
 mobs 0.06394132971763611
V 4
альными 0.06476493179798126
 Date 0.06443626433610916
);*/
 0.060840755701065063
� 0.0606062114238739
 upload 0.05983993038535118
V 5
 DAN 0.06578494608402252
 göç 0.06571841984987259
 SCR 0.06547516584396362
 extrapol 0.06267419457435608
ispers 0.06113431975245476
V 6
 Is 0.09318181872367859
 Does 0.08141963183879852
classmethod 0.07115177065134048
 IS 0.06131875514984131
直 0.05965099111199379
V 7
塔 0.06828374415636063
uslim 0.06619598716497421
Is 0.0

In [4]:
layer = 0
for i in range(32):
    # sa = model.get_submodule(f"model.model.layers.{layer}.mlp.up_proj")
    sa = model.get_submodule(f"model.model.layers.{layer}.self_attn.o_proj")
    lora_q = sa.lora_A.default.weight.data
    lorb_q = sa.lora_B.default.weight.data
    alpha = sa.scaling["default"]
    r = lora_q.size(0)
    dW = (lorb_q @ lora_q) * (alpha / r)
    u, s, vT = t.linalg.svd(dW.float(), full_matrices=False)
    v1 = vT[i]
    E = base.get_input_embeddings().weight                      
    sims = F.cosine_similarity(E, v1[None, :], dim=1)
    print("="*100)
    print(f"V {i}")
    topk = t.topk(sims, 5)
    for score, idx in zip(*topk):
        print(tokenizer.decode(idx.item()), float(score))
    print("="*100)

V 0
 виник 0.0681687444448471
/cs 0.06387200951576233
 Leads 0.06369868665933609
-vars 0.062205102294683456
ounter 0.06199835240840912
V 1
iciency 0.07117322832345963
).
 0.06410977989435196
Basically 0.06407978385686874
 rece 0.06357797235250473
 criticize 0.06150646135210991
V 2
ajes 0.06511102616786957
 Void 0.06270191073417664
 것입니다 0.060751378536224365
uish 0.06052376329898834
Numbers 0.06029621511697769
V 3
 calibrated 0.07164453715085983
isy 0.06834480166435242
)=" 0.06814742088317871
보고 0.0659969374537468
(console 0.06595347821712494
V 4
stopped 0.07330770045518875
adult 0.06534864008426666
 composition 0.06183508783578873
 inquiry 0.06077885255217552
 Composition 0.060759447515010834
V 5
Callback 0.06150611490011215
_repr 0.06042110547423363
'? 0.059667378664016724
 title 0.059055428951978683
urent 0.058872874826192856
V 6
']

 0.07059349864721298
 dreaming 0.06785692274570465
alarına 0.06499355286359787
 openid 0.06283324956893921
wicklung 0.061962734907865524
V 7
izacao 0.07